## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Current Description,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Atuona,PF,2021-05-31 15:24:18,scattered clouds,-9.8000,-139.0333,78.66,70,25,11.88
1,1,Cape Town,ZA,2021-05-31 15:32:31,light rain,-33.9258,18.4232,60.28,84,75,3.00
2,2,Yellowknife,CA,2021-05-31 15:21:33,few clouds,62.4560,-114.3525,47.32,63,20,5.01
3,3,Punta Arenas,CL,2021-05-31 15:32:18,clear sky,-53.1500,-70.9167,32.02,80,0,3.47
4,4,Roald,NO,2021-05-31 15:26:34,clear sky,62.5833,6.1333,74.16,72,0,13.80


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is min temp?"))
max_temp = float(input("What is max temp?"))


What is min temp?80
What is max temp?85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

filter_city_data_df = city_data_df.loc[(city_data_df["Max Temp"]>= min_temp)& (city_data_df["Max Temp"]<= max_temp)]

filter_city_data_df

,City_ID,City,Country,Date,Current Description,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
8,8,Butaritari,KI,2021-05-31 15:24:24,scattered clouds,3.0707,172.7902,82.60,77,36,15.99
11,11,Piacabucu,BR,2021-05-31 15:34:00,scattered clouds,-10.4056,-36.4344,82.71,66,34,8.14
37,37,Benguela,AO,2021-05-31 15:27:45,clear sky,-12.5763,13.4055,83.68,48,5,7.18
38,38,Muros,ES,2021-05-31 15:26:07,scattered clouds,42.7762,-9.0603,82.90,61,34,4.32
41,41,Coaraci,BR,2021-05-31 15:34:07,overcast clouds,-14.6408,-39.5511,80.04,89,85,4.43
...,...,...,...,...,...,...,...,...,...,...,...
642,642,Kulhudhuffushi,MV,2021-05-31 15:37:35,overcast clouds,6.6221,73.0700,83.07,74,97,20.07
647,647,Houma,US,2021-05-31 15:37:37,clear sky,29.5958,-90.7195,82.99,65,1,10.36
668,668,Manoel Urbano,BR,2021-05-31 15:37:45,broken clouds,-8.8389,-69.2597,84.76,79,81,3.27
673,673,Zhuhai,CN,2021-05-31 15:36:14,moderate rain,22.2769,113.5678,84.11,83,40,12.17


In [5]:
# 4a. Determine if there are any empty rows.

filter_city_data_df.count()

City_ID                81
City                   81
Country                80
Date                   81
Current Description    81
Lat                    81
Lng                    81
Max Temp               81
Humidity               81
Cloudiness             81
Wind Speed             81
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = filter_city_data_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Butaritari,KI,82.60,scattered clouds,3.0707,172.7902,
11,Piacabucu,BR,82.71,scattered clouds,-10.4056,-36.4344,
37,Benguela,AO,83.68,clear sky,-12.5763,13.4055,
38,Muros,ES,82.90,scattered clouds,42.7762,-9.0603,
41,Coaraci,BR,80.04,overcast clouds,-14.6408,-39.5511,
43,Puerto Ayora,EC,80.56,broken clouds,-0.7393,-90.3518,
46,Aketi,CD,82.63,overcast clouds,2.7388,23.7833,
48,Ishigaki,JP,80.80,overcast clouds,24.3448,124.1572,
49,Veraval,IN,83.82,broken clouds,20.9000,70.3667,
50,San Patricio,US,84.07,scattered clouds,28.0170,-97.5169,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
from pprint import pprint

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params ["location"] = f'{lat},{lng}'
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    response = requests.get(base_url,params = params).json()
    
#     pprint(response)
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = response["results"][0]["name"]
    except:
        print("Missing Hotel")
        

Missing Hotel
Missing Hotel
Missing Hotel
Missing Hotel
Missing Hotel


In [9]:
# 7. Drop the rows where there is no Hotel Name.

hotel_df.head()
clean_hotel_df = hotel_df.dropna()

In [11]:
# 8a. Create the output File (CSV)

output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Butaritari,KI,82.60,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
11,Piacabucu,BR,82.71,scattered clouds,-10.4056,-36.4344,Piaçabuçu News
37,Benguela,AO,83.68,clear sky,-12.5763,13.4055,Hotel Praia Morena
38,Muros,ES,82.90,scattered clouds,42.7762,-9.0603,Apartment Av. Castelao
41,Coaraci,BR,80.04,overcast clouds,-14.6408,-39.5511,Pousada Alternativa Coaraci


In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 

marker_layer= gmaps.marker_layer(locations,info_box_content=hotel_info)

# 11b. Display the figure

fig = gmaps.figure()
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))